## Get Packages

In [1]:
using HTTP
using DataFrames
using JSON
using DataStructures
using JSONStat

## Get data from API

In [2]:
params = Dict("query" => [
        Dict(
            "code" => "Tid",
            "selection" => Dict(
                "filter" => "item",
                "values" => ["2022U34", "2022U35"]
            )
        )], "response" => Dict("format" => "json-stat2"))

Dict{String, Any} with 2 entries:
  "response" => Dict("format"=>"json-stat2")
  "query"    => Dict{String, Any}[Dict("selection"=>Dict{String, Any}("values"=…

In [3]:
url = "https://data.ssb.no/api/v0/no/table/03024/"

"https://data.ssb.no/api/v0/no/table/03024/"

In [4]:
r = HTTP.request("POST", url,
                 ["Content-Type" => "application/json"],
                 JSON.json(params))

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Cache-Control: private
Content-Type: application/json; charset=utf-8
Access-Control-Allow-Origin: *
Date: Fri, 16 Sep 2022 18:14:07 GMT
Content-Length: 1010
X-Varnish: 1046523028
Age: 0
Via: 1.1 varnish (Varnish/6.0)
Content-Security-Policy: upgrade-insecure-requests
Expect-CT: enforce, max-age=3600
Referrer-Policy: no-referrer-when-downgrade
Strict-Transport-Security: max-age=86400
X-Content-Type-Options: nosniff
X-Frame-Options: SAMEORIGIN
X-XSS-Protection: 1; mode=block
X-Cache: MISS
X-UA-Compatible: IE=Edge
Set-Cookie: X-Identity=37702078.525; Path=/; Secure; HttpOnly
Accept-Ranges: bytes
Connection: keep-alive
Set-Cookie: ssbno=ffffffffc2a8b2e245525d5f4f58455e445a4a423660;expires=Fri, 16-Sep-2022 19:14:08 GMT;path=/;secure;httponly

{"class":"dataset","label":"03024: Eksport av oppalen laks, etter varegruppe, statistikkvariabel og uke","source":"Statistisk sentralbyrå","updated":"2022-09-14T06:00:00Z","id":["VareGrupper2","ContentsCode",

In [5]:
bodyjs = JSON.parse(String(r.body))

Dict{String, Any} with 11 entries:
  "label"     => "03024: Eksport av oppalen laks, etter varegruppe, statistikkv…
  "role"      => Dict{String, Any}("time"=>Any["Tid"], "metric"=>Any["ContentsC…
  "id"        => Any["VareGrupper2", "ContentsCode", "Tid"]
  "value"     => Any[24160, 24010, 64.62, 61.94, 719, 585, 86.83, 87.02]
  "version"   => "2.0"
  "updated"   => "2022-09-14T06:00:00Z"
  "class"     => "dataset"
  "dimension" => Dict{String, Any}("ContentsCode"=>Dict{String, Any}("label"=>"…
  "size"      => Any[2, 2, 2]
  "extension" => Dict{String, Any}("px"=>Dict{String, Any}("infofile"=>"None", …
  "source"    => "Statistisk sentralbyrå"

## Process data

In [55]:
ident = bodyjs["id"]

3-element Vector{Any}:
 "VareGrupper2"
 "ContentsCode"
 "Tid"

In [56]:
data = bodyjs["value"]

8-element Vector{Any}:
 24160
 24010
    64.62
    61.94
   719
   585
    86.83
    87.02

In [57]:
dim = bodyjs["dimension"]

Dict{String, Any} with 3 entries:
  "ContentsCode" => Dict{String, Any}("label"=>"statistikkvariabel", "link"=>Di…
  "Tid"          => Dict{String, Any}("label"=>"uke", "category"=>Dict{String, …
  "VareGrupper2" => Dict{String, Any}("label"=>"varegruppe", "category"=>Dict{S…

In [58]:
dsize = bodyjs["size"]

3-element Vector{Any}:
 2
 2
 2

In [63]:
matrix = reshape(data, dsize[1], dsize[2], dsize[3])

2×2×2 Array{Any, 3}:
[:, :, 1] =
 24160  64.62
 24010  61.94

[:, :, 2] =
 719  86.83
 585  87.02

### Adjust so we have julia indexing

In [8]:
for i in ident
    for (key, val) in dim[i]["category"]["index"]
        val = val+1
        dim[i]["category"]["index"][key] = val
    end
end

### Adjust internal dictionary according to the index

In [10]:
for i in ident
    rear = Dict()
    for key in keys(dim[i]["category"]["index"])
        idx = dim[i]["category"]["index"][key]
        vgr = dim[i]["category"]["label"][key]
        rear[idx] = vgr
    end
    dim[i]["category"]["indeks"] = rear
end

### Prepare and make Dataframe

In [53]:
tlabel = dim["Tid"]["label"]
vlabel = dim["VareGrupper2"]["label"]
clabel = dim["ContentsCode"]["label"]
valcol = "verdi"

mdf = Dict(tlabel => [], vlabel => [], clabel => [], valcol => [])

mdf = DataFrame(mdf)

,statistikkvariabel,uke,varegruppe,verdi
,Any,Any,Any,Any


In [54]:
for (tk, tv) in dim["Tid"]["category"]["indeks"]
    for (vk, vv) in dim["VareGrupper2"]["category"]["indeks"]
        for (ck, cv) in dim["ContentsCode"]["category"]["indeks"]
            push!(mdf,[tv, vv, cv, matrix[tk, ck, vk]])
        end
    end
end
mdf

,statistikkvariabel,uke,varegruppe,verdi
,Any,Any,Any,Any
1,2022U35,Frosen oppalen laks,Kilopris (kr),87.02
2,2022U35,Frosen oppalen laks,Vekt (tonn),585
3,2022U35,Fersk oppalen laks,Kilopris (kr),61.94
4,2022U35,Fersk oppalen laks,Vekt (tonn),24010
5,2022U34,Frosen oppalen laks,Kilopris (kr),86.83
6,2022U34,Frosen oppalen laks,Vekt (tonn),719
7,2022U34,Fersk oppalen laks,Kilopris (kr),64.62
8,2022U34,Fersk oppalen laks,Vekt (tonn),24160


In [45]:
dim["Tid"]["category"]["indeks"]

Dict{Any, Any} with 2 entries:
  2 => "2022U35"
  1 => "2022U34"

In [46]:
dim["VareGrupper2"]["category"]["indeks"]

Dict{Any, Any} with 2 entries:
  2 => "Frosen oppalen laks"
  1 => "Fersk oppalen laks"

In [47]:
dim["ContentsCode"]["category"]["indeks"]

Dict{Any, Any} with 2 entries:
  2 => "Kilopris (kr)"
  1 => "Vekt (tonn)"